In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/fairness-variance


In [4]:
import pandas as pd

from virny.datasets.data_loaders import CompasDataset, ACSIncomeDataset
from virny.utils.protected_groups_partitioning import create_test_protected_groups

## Folktables Income

In [5]:
age_range = [i for i in range(30, 46)]
sensitive_attributes_dct = {'SEX': '1', 'RAC1P': '1', 'AGEP': age_range, 'SEX & RAC1P': None, 'RAC1P & AGEP': None}

In [6]:
data_loader = ACSIncomeDataset(state=['GA'], year=2018, with_nulls=False, subsample_size=20_000, subsample_seed=42)
data_loader.full_df.head()

acs_data.head()


,SEX,PINCP
0,1,279000.0
1,2,23000.0
2,2,15100.0
3,1,31200.0
4,2,9400.0
5,1,62000.0
6,1,106000.0
7,1,43000.0
8,2,22000.0
9,2,100000.0


,SCHL,COW,MAR,OCCP,POBP,RELP,SEX,RAC1P,AGEP,WKHP
0,23,7,3,230,36,0,1,1,55,55.0
1,16,1,5,4110,13,2,2,1,20,35.0
2,16,4,3,4130,51,0,2,1,59,30.0
3,18,4,1,4020,13,0,1,2,43,40.0
4,14,1,1,8300,20,1,2,2,33,20.0


In [7]:
data_loader.full_df.shape

(20000, 10)

In [8]:
data_loader.full_df['SEX'].value_counts()

1    10291
2     9709
Name: SEX, dtype: int64

In [9]:
test_protected_groups = create_test_protected_groups(data_loader.X_data, data_loader.X_data, sensitive_attributes_dct)

In [10]:
for col_name in test_protected_groups.keys():
    print(f'{col_name}: {test_protected_groups[col_name].shape[0]}')

SEX_priv: 10291
SEX_dis: 9709
RAC1P_priv: 13535
RAC1P_dis: 6465
AGEP_priv: 6324
AGEP_dis: 13676
SEX&RAC1P_priv: 7271
SEX&RAC1P_dis: 3445
RAC1P&AGEP_priv: 4122
RAC1P&AGEP_dis: 4263


In [11]:
test_protected_groups.keys()

dict_keys(['SEX_priv', 'SEX_dis', 'RAC1P_priv', 'RAC1P_dis', 'AGEP_priv', 'AGEP_dis', 'SEX&RAC1P_priv', 'SEX&RAC1P_dis', 'RAC1P&AGEP_priv', 'RAC1P&AGEP_dis'])

In [20]:
for col_name in test_protected_groups.keys():
    proportion = test_protected_groups[col_name].shape[0] / data_loader.X_data.shape[0]
    print(f'{col_name}: {round(proportion, 3)}')

SEX_priv: 0.515
SEX_dis: 0.485
RAC1P_priv: 0.677
RAC1P_dis: 0.323
AGEP_priv: 0.316
AGEP_dis: 0.684
SEX&RAC1P_priv: 0.364
SEX&RAC1P_dis: 0.172
RAC1P&AGEP_priv: 0.206
RAC1P&AGEP_dis: 0.213


In [21]:
for col_name in test_protected_groups.keys():
    filtered_df = data_loader.y_data.iloc[test_protected_groups[col_name].index].copy(deep=True)
    base_rate = filtered_df[filtered_df == 1].shape[0] / filtered_df.shape[0]
    print(f'{col_name}: {round(base_rate, 3)}')

SEX_priv: 0.423
SEX_dis: 0.276
RAC1P_priv: 0.392
RAC1P_dis: 0.267
AGEP_priv: 0.386
AGEP_dis: 0.336
SEX&RAC1P_priv: 0.473
SEX&RAC1P_dis: 0.235
RAC1P&AGEP_priv: 0.433
RAC1P&AGEP_dis: 0.251


In [14]:
data_loader.y_data.iloc[test_protected_groups['SEX_priv'].index].head()

0    1
3    0
5    1
6    1
7    0
Name: PINCP, dtype: int64

In [15]:
data_loader.y_data.iloc[test_protected_groups['SEX_priv'].index].dtype

dtype('int64')

In [16]:
filtered_df = data_loader.y_data.iloc[test_protected_groups['SEX_priv'].index].copy(deep=True)
base_rate = filtered_df[filtered_df == 1].shape[0] / filtered_df.shape[0]

In [19]:
base_rate

0.4230881352638228

In [17]:
filtered_df[filtered_df == 1].shape[0]

4354

In [18]:
filtered_df.shape[0]

10291